In [ ]:
from Processor.Preprocessor import PreProcessing
from Model.AnomalyTransformer import AnomalyTransformer
from Model.EarlyStopping import EarlyStopping

from torch.nn.parameter import Parameter
import torch
import math
from torch import nn
from torch.nn import functional as F

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
Path = ''
train_dataloader,valid_dataloader,test_dataloader = PreProcessing(Path = Path)

In [ ]:
EPOCH = 1000
lambda_value = 3
window = 100
model = AnomalyTransformer(win_size = window, e_layers=6,enc_in = 4, c_out = 4).to(device)
es = EarlyStopping()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)
criterion = nn.MSELoss()

In [ ]:
import time
print("======================TRAIN MODE======================")



def my_kl_loss(p, q):
    res = p * (torch.log(p + 0.0001) - torch.log(q + 0.0001))
    return torch.mean(torch.sum(res, dim=-1), dim=1)

win_size=window
k = lambda_value

time_now = time.time()


for epoch in range(EPOCH):
    iter_count = 0
    loss1_list = []
    valid_list = []
    
    epoch_time = time.time()
    #train
    model.train()
    for i, (input_data,_) in enumerate(train_dataloader):

        optimizer.zero_grad()
        iter_count += 1
        input = input_data.float().to(device)
    
        output, series, prior, _ = model(input)

        # calculate Association discrepancy
        series_loss = 0.0
        prior_loss = 0.0
        for u in range(len(prior)):
            series_loss += (torch.mean(my_kl_loss(series[u], (
                    prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                           win_size)).detach())) + torch.mean(
                my_kl_loss((prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                                   win_size)).detach(),
                           series[u])))
            prior_loss += (torch.mean(my_kl_loss(
                (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                        win_size)),
                series[u].detach())) + torch.mean(
                my_kl_loss(series[u].detach(), (
                        prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                               win_size)))))
        series_loss = series_loss / len(prior)
        prior_loss = prior_loss / len(prior)

        rec_loss = criterion(output, input)

        loss1_list.append((rec_loss - k * series_loss).item())
        loss1 = rec_loss - k * series_loss
        loss2 = rec_loss + k * prior_loss
        
        
        # Minimax strategy
        loss1.backward(retain_graph=True)
        loss2.backward()
        optimizer.step()

    #valid
    model.eval()
    with torch.no_grad():
        for idx, (input_data,_) in enumerate(valid_dataloader):
            input = input_data.float().to(device)
            output, series, prior, _ = model(input)

            # calculate Association discrepancy
            series_loss = 0.0
            prior_loss = 0.0
            for u in range(len(prior)):
                series_loss += (torch.mean(my_kl_loss(series[u], (
                        prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                               win_size)).detach())) + torch.mean(
                    my_kl_loss((prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                                       win_size)).detach(),
                               series[u])))
                prior_loss += (torch.mean(my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                            win_size)),
                    series[u].detach())) + torch.mean(
                    my_kl_loss(series[u].detach(), (
                            prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                                   win_size)))))
            series_loss = series_loss / len(prior)
            prior_loss = prior_loss / len(prior)

            rec_loss = criterion(output, input)

            valid_list.append((rec_loss - k * series_loss).item())
            valid_loss = rec_loss - k * series_loss

        
    train_loss = np.average(loss1_list)
    es(valid_loss,model)
    if es.early_stop:
        print('Early Stopping')
        break
    print(f"Epoch: {epoch + 1} | Train Loss: {train_loss:.4f} | Valid Loss {valid_loss:.4f}")
    

In [ ]:
def anomaly_test(dataloader):
    temperature = 50
    test_energy = []
    test_labels = []
    model.eval()
    criterion = nn.MSELoss()
    win_size = 100

    for i, (input_data,labels) in enumerate(dataloader):
        input = input_data.float().to(device)
        output, series, prior, _ = model(input)
        loss = torch.mean(criterion(input, output), dim=-1)
        
        series_loss = 0.0
        prior_loss = 0.0
        for u in range(len(prior)):
            if u == 0:
                series_loss = my_kl_loss(series[u], (
                        prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                               win_size)).detach()) * temperature
                prior_loss = my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                            win_size)),
                    series[u].detach()) * temperature
            else:
                series_loss += my_kl_loss(series[u], (
                        prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                               win_size)).detach()) * temperature
                prior_loss += my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1,
                                                                                            win_size)),
                    series[u].detach()) * temperature
        
        metric = torch.softmax((-series_loss - prior_loss), dim=-1)
        cri = metric * loss
        cri = cri.detach().cpu().numpy()
        
        test_energy.append(cri)
        test_labels.append(labels)
        
        
        
        
        
    test_energy = np.concatenate(test_energy, axis=0).reshape(-1)
    test_energy = np.array(test_energy)
    test_labels = np.concatenate(test_labels, axis=0).reshape(-1)
    test_labels = np.array(test_labels)
    
    
    
    return test_energy, test_labels

In [ ]:
test_eg, test_la= anomaly_test(test_dataloader)

In [ ]:
def max_value(list1):
    list1 = list1.reshape(-1,100)
    max_value_list=[]
    for i in range(len(list1)):
        max_value= max(list1[i])
        max_value_list.append(max_value)
    max_value_list = np.array(max_value_list)
    return max_value_list

In [ ]:
mv_test = max_value(test_eg)
mv_tela = np.concatenate((mv_test.reshape(-1,1),test_la.reshape(-1,1)), axis = 1)
mv_tela = pd.DataFrame(mv_tela,columns = ['mv','la'])

In [ ]:
train_eg,test_la = anomaly_test(train_dataloader)
mv_train = max_value(train_eg)

In [ ]:
Q1 = np.percentile(mv_tela.mv[mv_tela.la==0], 25)
Q3 = np.percentile(mv_tela.mv[mv_tela.la==0], 75)
IQR = Q3 -Q1
threshold = Q3 + (1.0 * IQR)
print(f'Th : {threshold:.5f} | Q1 : {Q1:.5f} Q2 : {Q3:.5f} IQR : {IQR:.5f}')

In [ ]:
pred = []
for v,la in zip(mv_tela.mv,mv_tela.la):
    if v < threshold :
        pred.append(0)
    else:
        pred.append(1)

In [ ]:
y_true = []
y_pred = []
for i in range(4):
    if i == 0:
        for v in mv_tela[mv_tela.la==i].mv:
            y_true.append(i)
            if v <= threshold:
                y_pred.append(i)
                
            else :
                y_pred.append(2)
    else:
        for v in mv_tela[mv_tela.la==i].mv:
            y_true.append(i)
            if v >= threshold:
                y_pred.append(i)
            else:
                y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
sns.heatmap(cm, annot = True, cmap = 'Blues',fmt='.0f')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Anomaly Transformer')

In [ ]:
TN=0
TP=0
FP=0
FN=0
for pr,la in zip(pred,mv_tela.la):
    if pr == 0 and la == 0:
        TN += 1
    elif pr != 0 and la != 0:
        TP += 1
    elif pr != 0 and la == 0:
        FP += 1
    else:
        FN +=1
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 2/((1/precision)+(1/recall))
print(f'precision : {precision*100:.3f}')
print(f'recall : {recall*100:.3f}')
print(f'F1-score : {f1*100:.3f}')
